In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
findspark.find()

'C:\\spark-3.3.1-bin-hadoop2'

# method 1 reading csv file into spark session directly from local

In [4]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, cast, count, isnan, isnull, when


In [5]:
spark = SparkSession.builder.appName("diabet").master("local[2]").getOrCreate()

In [6]:
spark

In [7]:
#spark.sparkContext.setSystemProperty()

In [8]:
rawd = spark.read.csv("../drang/Downloads/misc/diabetes.csv", header=True)
df = spark.range(0,20)
print(df.rdd.getNumPartitions())

2


In [9]:
rawd.rdd.getNumPartitions()

2

#  method 2 creating DB from csv local file and updating to server

In [10]:
#method 2 creating DB from csv local file and updating to server
import pandas as pd

data = pd.read_csv(r"C:\Users\drang\Downloads\misc\diabetes.csv")


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\drang\\Downloads\\misc\\diabetes.csv'

In [ ]:
import mysql.connector

db_connection = mysql.connector.connect(user="root", password="A")
db_cursor = db_connection.cursor(buffered=True)
#db_cursor.execute("CREATE DATABASE DDB;")
db_cursor.execute("USE DDB;")



In [ ]:
db_cursor.execute("CREATE TABLE diabetes1(Pregnancies INT, Glucose INT, \
                   BloodPressure INT, SkinThickness INT, BMI FLOAT, \
                   Insulin INT, DiabetesPedigreeFunction FLOAT, Age FLOAT, Outcome INT);")


In [ ]:
#create structure for sql
wtuples = list(data.itertuples(index=False, name=None))
tuples_string = ",".join(["(" + ",".join([str(w) for w in wt]) + ")" for wt in wtuples])

In [ ]:
db_cursor.execute("INSERT INTO diabete(Pregnancies, Glucose, BloodPressure,\
                   SkinThickness, Insulin, BMI, DiabetesPedigreeFunction, Age, Outcome) VALUES " + tuples_string + ";")
db_cursor.execute("FLUSH TABLES;")

In [ ]:
db_cursor.execute("SELECT * FROM diabete;")

In [ ]:
spark.stop()

# method 2-2pt read data from mysql server into spark session

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.config("spark.jars", r"C:\spark-3.3.1-bin-hadoop2\jars\mysql-connector-j-8.0.31.jar") \
.config("spark.sql.sources.bucketing.enabled", True) \
.config("spark.sql.shuffle.partitions", "500") \
.config("spark.default.parallelism", "1") \
.config("spark.driver.extraClassPath", r"C:\spark-3.3.1-bin-hadoop2\jars\mysql-connector-j-8.0.31.jar") \
.config("spark.executor.extraClassPath", r"C:\spark-3.3.1-bin-hadoop2\jars\mysql-connector-j-8.0.31.jar") \
.config("spark.executor.extraLibrary", r"C:\spark-3.3.1-bin-hadoop2\jars\mysql-connector-j-8.0.31.jar") \
    .master("local").appName("PySpark_MySQL_test").enableHiveSupport().getOrCreate()
#last 2 configs for cluster mode instead of local mode  
    
    
df = spark.read.format("jdbc").option("url", "jdbc:mysql://localhost:3306/DDB") \
    .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "diabete") \
    .option("user", "root").option("password", "Art8870496550!@#").load() 

In [ ]:
df.show()

In [ ]:
print(df.rdd.getNumPartitions())
df.explain()

In [ ]:
df.write.bucketBy(10,"BMI")#.saveAsTable("bucketed_large_table_5")
#df.rdd.collect()
#bucketing usd for partition not working here
df.show()
spark.sql("SELECT * FROM bucketed_large_table_5").show()


In [ ]:
az=df.repartition(100)
df.write.option("header",True) \
        .partitionBy("Pregnancies") \
        .mode("overwrite") \
        .csv(r"C:\Users\drang\Downloads\misc\diabetes.csv")

In [ ]:
columnname = df.columns
#print(az.rdd.getNumPartitions())
df.explain()

In [ ]:
df.printSchema()

In [ ]:
rawf = df.select(*(col(c).cast('float').alias(c) for c in df.columns))

In [ ]:
rawf.printSchema()

In [ ]:
rawf.select([count(when(col(c).isNull(),c)).alias(c) for c in rawf.columns]).show()

In [ ]:
#if theres null use imputer
from pyspark.ml.feature import Imputer
imputer = Imputer(inputCol='Age', outputCol='newage')
model = imputer.fit(rawf)
data = model.transform(rawf)

In [ ]:
data.rdd.getNumPartitions()
d = data.repartition(5)
d.rdd.getNumPartitions()


In [ ]:

colsi = rawf.drop(data.Outcome)

colsi = colsi.columns
colsi

In [ ]:
from pyspark.ml.feature import VectorAssembler
n= VectorAssembler(inputCols=colsi, outputCol="feature")
newd = n.transform(rawf)
newd.show(truncate=False)

In [ ]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol='feature', outputCol="scaledfeat")
model = scaler.fit(newd)
scaledd = model.transform(newd)


In [ ]:
newdata = scaledd.select('scaledfeat','Outcome')

In [ ]:
newdata.show(truncate=False)

In [ ]:
from pyspark.ml.feature import ChiSqSelector
chi = ChiSqSelector(numTopFeatures=6, outputCol="selectedcols", featuresCol="scaledfeat", labelCol="Outcome")
Sdata = chi.fit(newdata).transform(newdata)

In [ ]:
Sdata.show(truncate=False)

In [ ]:
columnname #3rd and 7th- BP & DIABETESPEDIGREE REMOVED BY CHISQ FEAT SELECTOR

In [ ]:
Ssdata= Sdata.drop("scaledfeat")
Ssdata.show(truncate=False)

In [ ]:
#traib test split
train, test = newdata.randomSplit([0.7,0.3])
test.count()
newdata.show()

In [ ]:
from pyspark.ml.classification import LogisticRegression, GBTClassifier, RandomForestClassifier

In [ ]:
logreg = LogisticRegression(labelCol="Outcome", featuresCol="scaledfeat", maxIter=500, regParam=0.2, elasticNetParam=0.8)
model = logreg.fit(train)
preds = model.transform(test)
evals = model.evaluate(test)

In [ ]:
preds.show()
"""16 - p1 30
12 -p2 20
11 - p4 15"""
print(evals.accuracy, evals.areaUnderROC)


In [ ]:
import time
start= time.time()
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

rf = RandomForestClassifier(labelCol="Outcome", featuresCol="scaledfeat", numTrees=20)


paramGrid = ParamGridBuilder() \
  .addGrid(rf.maxDepth, [5, 10, 15, 20]) \
  .addGrid(rf.maxBins, [25, 30]) \
  .build()
paramGrid
"""trainn = train.repartition(4)
testt = test.repartition(4)"""
print(train.rdd.getNumPartitions())
evaluator=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="Outcome", metricName="areaUnderROC")
tvs = TrainValidationSplit(
  estimator=rf,
  estimatorParamMaps=paramGrid,
  evaluator=evaluator,
  trainRatio=0.8,parallelism=4, seed=53)

tvs_m=tvs.fit(train)
preds = tvs_m.transform(test)
print("roc",evaluator.evaluate(tvs_m.transform(test)))
#print(evals.accuracy, evals.areaUnderROC)
print("time:", time.time() - start, "secs")

In [ ]:
"""from pyspark.mllib.evaluation import BinaryClassificationMetrics
#convert to rdd cox of mllib pkg
predsa = preds.select('Outcome','prediction').rdd.map(lambda row: (float(row['probability'][1]), float(row['Outcome'])))
metrics = BinaryClassificationMetrics(predsa)"""

In [ ]:
"""print("AUC", metrics.areaUnderPR)"""

In [ ]:
predy = preds.select('Outcome','rawPrediction', 'prediction')

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
metric = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", labelCol="Outcome", metricName="areaUnderROC")
lrroc=metric.evaluate(predy)
lrroc

In [ ]:
metrics = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="Outcome", metricName="accuracy")
lracc = metrics.evaluate(predy)
lracc

In [ ]:

rf = RandomForestClassifier(labelCol="Outcome", featuresCol="scaledfeat", numTrees=20)
rfmod = rf.fit(train)
pred = rfmod.transform(test)

In [ ]:
predy = preds.select('Outcome','rawPrediction', 'prediction')

In [ ]:
rfroc = metric.evaluate(predy)

In [ ]:
rfacc = metrics.evaluate(predy)

In [ ]:
ml = GBTClassifier(labelCol="Outcome", featuresCol=("scaledfeat"), maxDepth=10, maxIter=10)
model = ml.fit(train)


In [ ]:
preds = model.transform(test)

#convert SparseVector -> DenseVector  which error occured during chisq selector

In [ ]:
predy = preds.select('Outcome','rawPrediction', 'prediction')

gbtroc=metric.evaluate(predy)

In [ ]:
gbtacc = metrics.evaluate(predy)

In [ ]:
roc = [lrroc, rfroc, gbtroc]
acc = [lracc, rfacc, gbtacc]
print(f"lrroc:{lrroc},  rfroc:{rfroc}, gbtroc:{gbtroc}")
print(f"lracc:{lracc},  rfacc:{rfacc}, gbtacc:{gbtacc}")

In [ ]:
from pyspark.mllib.stat import Statistics
n = newdata.drop("Outcome")
n = newdata.select(newdata.scaledfeat).rdd.map(lambda row : row[0:])
summary = Statistics.colStats(n)

summary.mean() #variance, norm

In [ ]:
correlat = Statistics.corr(n, method='pearson')
import pandas as pd
pddf= pd.DataFrame(correlat, columns=columnname[:-1], index=columnname[:-1])
pddf


In [ ]:
from pyspark.ml.stat import Correlation
n = newd.drop('Outcome')
n = n.drop('features')
summary = Correlation()
print(n)
columnname=columnname[:-1]
a=summary.corr(dataset=n.feature, column=columnname) #variance, norm
a

In [ ]:
n.show()